# Test 010424


In [ ]:
import pandas as pd

In [ ]:
def read_data(file_path):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    with open(file_path, "r") as f:
        for line in f.readlines():
            # 过长的评论文本会超出io限制报错。 暂时忽略
            try:
                # 字符串预处理
                # 布尔替换成python的习惯
                str_text = line.replace("true", "True")
                str_text = str_text.replace("false", "False")
                # 转成字典形式
                raw_sample = eval(str_text)
                data.append([raw_sample['reviewerID'],
                             raw_sample['asin'],
                             raw_sample['overall'],
                             raw_sample['reviewText']])
            except:
                pass
    return data


In [ ]:
data = read_data("./data/raw/Beauty_5.json")
data_df = pd.DataFrame(data)
data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']
data = data_df["reviewText"].tolist()

In [ ]:
import torch
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel

model_name='bert-base-uncased'
# Khởi tạo tokenizer và model BERT
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)
def extract_bert_features(text, model, tokenizer, output_dim=10):


    # Tokenize văn bản và thêm token '[CLS]' và '[SEP]'
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

    # Feed-forward văn bản qua model BERT để trích xuất đặc trưng
    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy đặc trưng từ lớp embedding (lớp cuối cùng trước lớp softmax)
    features = outputs.last_hidden_state

    # Pooling các đặc trưng từ các token thành một vector đặc trưng (ở đây ta sử dụng mean pooling)
    pooled_features = torch.mean(features, dim=1)  # Kích thước: (1, hidden_size)

    # Giảm chiều feature đầu ra bằng cách sử dụng một lớp Linear
    linear_layer = torch.nn.Linear(pooled_features.shape[1], output_dim)
    reduced_features = linear_layer(pooled_features)

    return reduced_features

# Tính cosine similarity
def cosine_similarity(vec1, vec2):
    return F.cosine_similarity(vec1, vec2).item()

# Tính Euclidean distance
def euclidean_distance(vec1, vec2):
    return torch.dist(vec1, vec2).item()



In [ ]:
# # Sử dụng hàm để trích xuất đặc trưng từ văn bản
# text = "Hello"
# features = extract_bert_features(text, model, tokenizer)
# print(features)

# # Sử dụng hàm để trích xuất đặc trưng từ văn bản
# text = "Hello"
# features = extract_bert_features(text, model, tokenizer)
# print(features)

# Ví dụ về sử dụng
text1 = "Hello, how are you?"
text2 = "Hi, how are you doing?"
features1 = extract_bert_features(text1, model, tokenizer)
features2 = extract_bert_features(text2, model, tokenizer)


print("Cosine similarity:", cosine_similarity(features1, features2))
print("Euclidean distance:", euclidean_distance(features1, features2))

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, Dataset

# Tạo dataset đơn giản
train_data = [
    {'text': 'This is a positive sentence.', 'label': 1},
    {'text': 'I love using BERT for natural language processing tasks.', 'label': 1},
    {'text': 'Negative sentiment is not good for the mood.', 'label': 0},
    {'text': 'The quick brown fox jumps over the lazy dog.', 'label': 1},
    {'text': 'I am not feeling well today.', 'label': 0}
]

# Chuẩn bị dữ liệu huấn luyện
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]['text']
        label = self.data[idx]['label']

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].squeeze(0)
        attention_mask = encoding['attention_mask'].squeeze(0)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': torch.tensor(label)
        }

# Chuẩn bị mô hình BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Fine-tuning mô hình
train_dataset = CustomDataset(train_data, tokenizer)
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
criterion = torch.nn.CrossEntropyLoss()

num_epochs = 3
model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for batch in train_dataloader:
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['label']

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")

# Lưu mô hình sau khi fine-tune
model.save_pretrained("fine_tuned_bert_model")


In [ ]:
# Load mô hình và tokenizer đã fine-tune
model = BertForSequenceClassification.from_pretrained("fine_tuned_bert_model")
# Load tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Chuẩn bị câu văn bản để test
text = "This is a positive sentence."

# Tokenize câu văn bản
inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

# Phân loại hoặc trích xuất đặc trưng từ câu văn bản
with torch.no_grad():
    outputs = model(**inputs)

# Nếu muốn phân loại, sử dụng logits
logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
print("Predicted class:", predicted_class)

# Nếu muốn trích xuất đặc trưng, sử dụng đặc trưng từ lớp embedding (lớp cuối cùng trước lớp softmax)
features = outputs.last_hidden_state.mean(dim=1).squeeze(0)  # Sửa đoạn này
print("Extracted features:", features)

In [ ]:
import pandas as pd

data = read_data("/content/drive/MyDrive/Recommandation System/AMAZON_FASHION_5.json")
data_df = pd.DataFrame(data)
data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']
data = data_df["reviewText"].tolist()
# In kết quả xuống hàng mỗi record
for record in data[:20]:  # Chỉ in 20 record đầu tiên
    print(record)

In [ ]:
from nltk.corpus import wordnet, stopwords
from collections import Counter
import re
import nltk

nltk.download('wordnet')
nltk.download('stopwords')

def preprocess_text(text):
    # Loại bỏ ký tự không cần thiết và chuyển đổi thành chữ thường
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    return text

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

def get_common_words(texts, n):
    # Tiền xử lý văn bản và loại bỏ từ dừng
    preprocessed_texts = [remove_stopwords(preprocess_text(text)) for text in texts]

    # Tách thành các từ riêng lẻ
    words = ' '.join(preprocessed_texts).split()

    # Đếm tần suất của các từ
    word_counts = Counter(words)

    # Chọn các từ có tần suất xuất hiện cao nhất
    common_words = [word for word, count in word_counts.most_common(n)]

    return common_words

def convert_records_to_dataset(records):
    dataset_words = []
    for record in records:
        # Tiền xử lý văn bản
        preprocessed_text = preprocess_text(record)

        # Loại bỏ các từ dừng
        filtered_words = remove_stopwords(preprocessed_text)

        # Thêm các từ vào dataset_words
        words = filtered_words.split()
        for word in words:
          dataset_words.append(word)

    return dataset_words

def semantic_similarity(word1, word2):
    # Tìm các synset (nhóm đồng nghĩa) của từ 1 và từ 2
    synsets1 = wordnet.synsets(word1)
    synsets2 = wordnet.synsets(word2)
    print(synsets1)
    print(synsets2)
    max_similarity = 0

    # Lặp qua các synset của từ 1 và từ 2 để tìm độ tương đồng cao nhất
    for synset1 in synsets1:
        for synset2 in synsets2:
            similarity = synset1.wup_similarity(synset2)  # Sử dụng phương pháp Wu-Palmer để tính độ tương đồng
            if similarity is not None and similarity > max_similarity:
                max_similarity = similarity

    return max_similarity
print(semantic_similarity("great", "best"))
# Ví dụ dữ liệu văn bản
dataset_words = convert_records_to_dataset(data[:20])
print(dataset_words)

# Lấy 10 từ xuất hiện nhiều nhất trong tập dữ liệu
common_words = get_common_words(data[:20], 10)

# Tìm các từ đồng nghĩa với từng từ phổ biến
synonyms_dict = {}
for word in common_words:
    synonyms = find_synonyms_in_dataset(word, dataset_words)
    synonyms_dict[word] = synonyms

# In kết quả
for word, synonyms in synonyms_dict.items():
    print(f"Synonyms of '{word}': {', '.join(synonyms)}")


In [ ]:
!pip install pytesseract
!pip install pillow==10.0.0
!pip install pillow-heif


In [ ]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract

# Tải xuống tập tin dữ liệu ngôn ngữ cho tiếng Việt
!wget https://github.com/tesseract-ocr/tessdata/raw/main/vie.traineddata -P /usr/share/tesseract-ocr/4.00/tessdata/



In [ ]:
import os

# Thiết lập biến môi trường TESSDATA_PREFIX
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata/'

In [ ]:
!pip install easyocr

In [ ]:
import pytesseract
from PIL import Image
import matplotlib.pyplot as plt
from pillow_heif import register_heif_opener
import easyocr


register_heif_opener()
def ocr_vietnamese_text(image_path):
    # Mở ảnh sử dụng Pillow
    img = Image.open(image_path)

    # Chuyển đổi ảnh sang định dạng PNG (nếu không phải là PNG)
    if img.format != "PNG":
        img = img.convert("RGB")  # Chuyển đổi sang ảnh RGB (định dạng phổ biến nhất)
        png_image_path = "converted_image.png"
        img.save(png_image_path, format="PNG")  # Lưu ảnh dưới dạng PNG

    else:
        png_image_path = image_path  # Sử dụng ảnh gốc nếu đã là định dạng PNG
    plt.imshow(img)
    plt.show()
    # Sử dụng Tesseract OCR để chuyển đổi ảnh thành văn bản
    text = pytesseract.image_to_string(png_image_path, lang='vie')

    return text

def ocr_vietnamese_and_english(image_path):
    # Đường dẫn đến ảnh
    reader = easyocr.Reader(['en', 'vi'])  # Chỉ định ngôn ngữ cần nhận dạng
    img = Image.open(image_path)
    # Chuyển đổi ảnh sang định dạng PNG (nếu không phải là PNG)
    if img.format != "PNG":
        img = img.convert("RGB")  # Chuyển đổi sang ảnh RGB (định dạng phổ biến nhất)
        png_image_path = "converted_image.png"
        img.save(png_image_path, format="PNG")  # Lưu ảnh dưới dạng PNG

    else:
        png_image_path = image_path  # Sử dụng ảnh gốc nếu đã là định dạng PNG
    plt.imshow(img)
    plt.show()

    # Nhận dạng văn bản từ ảnh
    result = reader.readtext(png_image_path)

    # Tạo một đoạn văn bản từ kết quả nhận dạng
    text = '\n'.join([entry[1] for entry in result])

    return text

# Đường dẫn đến ảnh cần chuyển đổi
image_path = "/content/drive/MyDrive/Recommandation System/IMG_9218.heic"

# Sử dụng hàm ocr_vietnamese_text để chuyển đổi ảnh sang văn bản
# text = ocr_vietnamese_text(image_path)
text = ocr_vietnamese_and_english(image_path)

# In văn bản được chuyển đổi
print(text)

# **Test 080424**

In [32]:
import pandas as pd
import json
import csv
import os

In [69]:
def read_data(file_path):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    with open(file_path, "r") as f:
        for line in f.readlines():
            # 过长的评论文本会超出io限制报错。 暂时忽略
            try:\
                # 字符串预处理
                # 布尔替换成python的习惯
                str_text = line.replace("true", "True")
                str_text = str_text.replace("false", "False")
                # 转成字典形式
                raw_sample = json.loads(str_text)
                print(raw_sample)
                data.append(raw_sample)
            except:
                print(line)
                pass
    return data

def read_and_split_data(file_path, batch_size=10000):
    """
       params:
           file_path: 文件路径
       return:
           data: 读取的数据列表，每行一条样本

    """
    data = []
    small_list_data = []
    count = 0
    with open(file_path, "r") as f:
        for line in f.readlines():
            if count < batch_size:
                # 过长的评论文本会超出io限制报错。 暂时忽略
                try:
                    # 字符串预处理
                    # 布尔替换成python的习惯
                    str_text = line.replace("true", "True")
                    str_text = str_text.replace("false", "False")
                    # 转成字典形式
                    raw_sample = eval(str_text)
                    data.append(raw_sample)
                    count += 1
                except:
                    pass
            else:
                small_list_data.append(data)
                data = []
                count = 0
    small_list_data.append(data)
    return small_list_data

def read_txt_file_to_list(file_path):
    records = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            records.append(line.strip())
    return records


def remove_duplicate_rows(df):
    # Loại bỏ các hàng trùng lặp dựa trên cột 'reviewText'
    df = df.drop_duplicates(subset=['reviewText', 'asin', 'reviewerID'])
    return df

def remove_duplicate_rows_ver2(df):
    # Loại bỏ các hàng trùng lặp dựa trên cột 'reviewText'
    df = df.drop_duplicates(subset=['reviewText', 'itemID', 'reviewerID'])
    return df

In [ ]:
json_file_path = "./DeepCGSR_triet/data/raw/Digital_Music_5.json"  # Thay đổi đường dẫn tới file JSON của bạn
output_directory = "./DeepCGSR_triet/data/DigitalMusic"  # Thay đổi đường dẫn thư mục lưu file CSV của bạn
chunk_size = 10000  # Kích thước của mỗi phần dữ liệu

small_list_data = read_and_split_data(json_file_path, chunk_size)
count_name = 0
for data in small_list_data:
    # Chuyển đổi dữ liệu thành DataFrame
    data_df = pd.DataFrame(data)
    print(data_df)
    # data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']

    # Lưu dữ liệu dưới dạng CSV
    output_file_path = os.path.join(output_directory, f"Small_Digital_Music_{len(data_df)}_{count_name}.json")
    data_df.to_json(output_file_path, orient='records', lines=True)

    print(f"Saved {len(data_df)} records to {output_file_path}")
    count_name += 1

In [71]:
data = pd.read_json("DeepCGSR_triet/data/DigitalMusic/Small_Digital_Music_10000_1.json", lines=True)
print(data)
data_df = pd.DataFrame(data, columns=['reviewerID', 'asin', 'overall', 'reviewText'])
print(data_df.head())
data_df.columns = ['reviewerID', 'asin', 'overall', 'reviewText']

data_rating = data_df["overall"].tolist()
data_review = data_df["reviewText"].tolist()
data_reviewerID = data_df["reviewerID"].tolist()
data_itemID = data_df["asin"].tolist()

      overall  verified   reviewTime      reviewerID        asin  \
0           5      True   08 7, 2012  A2AVMXNBKEL5T6  B000WOXPNS   
1           5      True  02 18, 2013  A1B7R4FDTNQZWE  B000WOT9K6   
2           5      True  02 13, 2013  A3AGJ0NQXOW8M1  B000WOT9K6   
3           5      True  12 22, 2012  A34UTNSRNF2AYB  B000WOT9K6   
4           5      True  12 19, 2014  A2TXKUC8SN53H7  B000WOT9K6   
...       ...       ...          ...             ...         ...   
9995        5     False   02 9, 2016  A1R8VKMP29VO4R  B00136NEAI   
9996        5      True  01 31, 2016  A3QQXGSVDG50AB  B00136NEAI   
9997        5      True  01 18, 2016  A16S5FX3Z55Q0X  B00136NEAI   
9998        5      True  12 16, 2015   APFPBV1WMGQEI  B00136NEAI   
9999        5      True  11 20, 2015  A3H4BQD3T1ME2A  B00136NEAI   

                          style     reviewerName  \
0     {'Format:': ' MP3 Music'}             Mitz   
1     {'Format:': ' MP3 Music'}             Lynn   
2     {'Format:': ' MP3 Mus

In [72]:
print(len(data_rating))
print(len(data_reviewerID))

10000
10000


In [73]:
import pandas as pd
# step01:
def defineRating(data_rating):
  T_v = []
  for rating in data_rating:
    if rating >= 4:
      T_v.append(1)
    else:
      T_v.append(-1)
  return T_v

T_v = defineRating(data_rating)

# Define the data for the dataframe
data = {'id': range(len(T_v)), 'T_v': T_v}

# Create the dataframe
df_rating = pd.DataFrame(data)

# Print the dataframe
print(df_rating)


        id  T_v
0        0    1
1        1    1
2        2    1
3        3    1
4        4    1
...    ...  ...
9995  9995    1
9996  9996    1
9997  9997    1
9998  9998    1
9999  9999    1

[10000 rows x 2 columns]


In [74]:
# Thêm vào dataframe df_rating cột C_v
df_rating['reviewerID'] = data_reviewerID
df_rating['asin'] = data_itemID
df_rating['overall'] = data_rating
df_rating['reviewText'] = data_review
# Loại bỏ các dòng có reviewText trùng lặp
# df_rating = remove_duplicate_rows(df_rating)
print(df_rating)

        id  T_v      reviewerID        asin  overall  \
0        0    1  A2AVMXNBKEL5T6  B000WOXPNS        5   
1        1    1  A1B7R4FDTNQZWE  B000WOT9K6        5   
2        2    1  A3AGJ0NQXOW8M1  B000WOT9K6        5   
3        3    1  A34UTNSRNF2AYB  B000WOT9K6        5   
4        4    1  A2TXKUC8SN53H7  B000WOT9K6        5   
...    ...  ...             ...         ...      ...   
9995  9995    1  A1R8VKMP29VO4R  B00136NEAI        5   
9996  9996    1  A3QQXGSVDG50AB  B00136NEAI        5   
9997  9997    1  A16S5FX3Z55Q0X  B00136NEAI        5   
9998  9998    1   APFPBV1WMGQEI  B00136NEAI        5   
9999  9999    1  A3H4BQD3T1ME2A  B00136NEAI        5   

                                             reviewText  
0     This is a great song.  It reminds me of rollin...  
1     I certainly enjoyed this item.  I would defini...  
2     IF YOU LOVE BACK , BACK, BACK, UMM BACK IN THE...  
3     This one brings my Switch collection in to for...  
4                                    

In [7]:

# ratings_grouped = df_rating.groupby('reviewerID')['overall'].agg(['min', 'max', list])
# print(ratings_grouped)
# filtered_users_max = ratings_grouped[ratings_grouped['max'] != 5.0]
# print(filtered_users_max)

# user_ratings = df_rating[df_rating['reviewerID'] == 'A2I9O5E0Q731GN']['overall'].tolist()
# print(user_ratings)




                min  max             list
reviewerID                               
A11QGZ39A7ZF0X  5.0  5.0  [5.0, 5.0, 5.0]
A24HQ2N7332W7W  5.0  5.0            [5.0]
A24W4W9E62FZP2  2.0  2.0            [2.0]
A2G90R2ZU6KU5D  5.0  5.0            [5.0]
A2UEO5XR3598GI  5.0  5.0            [5.0]
A3CIUOJXQ5VDQ2  5.0  5.0            [5.0]
A3H7T87S984REU  5.0  5.0            [5.0]
A3J034YH7UG4KT  1.0  1.0            [1.0]
A3SFRT223XXWF7  5.0  5.0            [5.0]
A7ID5H7FWLJHC   1.0  1.0            [1.0]
AYKOSAJTP5AVS   1.0  1.0            [1.0]
                min  max   list
reviewerID                     
A24W4W9E62FZP2  2.0  2.0  [2.0]
A3J034YH7UG4KT  1.0  1.0  [1.0]
A7ID5H7FWLJHC   1.0  1.0  [1.0]
AYKOSAJTP5AVS   1.0  1.0  [1.0]
[]


# SODCM

In [ ]:
from textblob import TextBlob


# Define function to extract a string's sentiment
find_sentiment = lambda text: TextBlob(text).sentiment[0]
# Create new column in dataframe
C_v = data_df["reviewText"].apply(find_sentiment)



In [ ]:
# Thêm vào dataframe df_rating cột C_v
df_rating['C_v'] = C_v
print(df_rating[df_rating['id'] == 98])

In [ ]:
# step02:
import math


def euclidean_distance(p1, p2):
  distance = math.sqrt((p2 - p1)**2)
  return distance
V_d = []
for ti, ci in zip(T_v, C_v):
  V_d.append(euclidean_distance(ti, ci))
  # V_d.append((ti + ci)/2)

In [ ]:
# Thêm vào dataframe df_rating cột V_d
df_rating['V_d'] = V_d
print(df_rating)

In [ ]:
# step03:
S_pos = []
S_neg = []

i = 0
for r in data_rating:
  if r >= 4:
    S_pos.append([i, r, V_d[i]])
  else:
    S_neg.append([i, r, V_d[i]])
  i = i + 1

print(len(S_pos))
print(len(S_neg))
element = None
i = 0
for item in S_pos:
    if item[0] == 25:
        element = item
        break
    i = i + 1
# for item in S_neg:
#     if item[0] == 25:
#         element = item
#         break
print(S_pos[17])
print(element)



In [ ]:
#step04:

def calculation_IQR(S, V_d, flag='pos'):
  # Sắp xếp dữ liệu
  sorted_data = sorted(V_d)

  # Tính Q1 và Q3
  q1_index = int(len(sorted_data) * 0.25)
  q3_index = int(len(sorted_data) * 0.75)
  Q1 = sorted_data[q1_index]
  Q3 = sorted_data[q3_index]

  # Tính IQR
  IQR = Q3 - Q1

  # Tính F_L và F_U
  F_L = Q1 - 1.5 * IQR
  F_U = Q3 + 1.5 * IQR

  if flag == 'pos':
    if F_L < 0:
      O_s =  Q3 + IQR
    else:
      O_s = F_U - IQR*F_L
  else:
    if F_U > 2:
      O_s = Q1 - IQR
    else:
      O_s = Q3 - IQR*F_U

  return O_s

O_Spos = calculation_IQR(S_pos, V_d, flag = 'pos')
O_Sneg = calculation_IQR(S_neg, V_d, flag = 'neg')

print(O_Spos)
print(O_Sneg)

In [ ]:
#step05:


# S_pos: danh sách các chỉ số của các phần tử thuộc S+
# S_neg: danh sách các chỉ số của các phần tử thuộc S-
# VD: danh sách các giá trị của biến VD
# OS_pos: giá trị OS+ như trong mã script
# OS_neg: giá trị OS- như trong mã script

# Khởi tạo danh sách O+ và O-
O_pos = [0] * len(S_pos)
O_neg = [0] * len(S_neg)

# print(S_pos)
# print(S_neg)
count = 0
# Điều chỉnh danh sách O+ và O- theo điều kiện trong mã script
for i, ri in enumerate(S_pos):
  print(i, ri)
  if ri[2] >= O_Spos:
    O_pos[i] = 'yes'
    count += 1
  else:
    O_pos[i] = 'no'

for i, ri in enumerate(S_neg):
    if ri[2] <= O_Sneg:
      O_neg[i] = 'yes'
      count += 1
    else:
      O_neg[i] = 'no'

print(count)
# In ra kết quả
# print("O+:", O_pos)
# print("O-:", O_neg)


In [ ]:
df = pd.DataFrame(S_pos, columns=['id', 'rating', 'V_d'])
df['O_pos'] = O_pos
#In ra các phần tử có giá trị 'yes'
print(df[df['O_pos'] == 'yes'])


In [ ]:
#step06:
# Hàm toggle để đảo ngược giá trị của một biến
def toggle(value):
  if value == -1:
    return 1
  else:
    return -1

# Khởi tạo danh sách TV với giá trị False cho mỗi phần tử
TV_new = T_v.copy()

count_1 = 0
for i, ri in enumerate(S_pos):
    if O_pos[i] == 'yes':
      count_1 += 1
      TV_new[ri[0]] = toggle(T_v[ri[0]])

for i, ri in enumerate(S_neg):
    if O_neg[i] == 'no':
      count_1 += 1
      TV_new[ri[0]] = toggle(T_v[ri[0]])

# print(TV_new)
# Tạo danh sách D* bằng cách nối S+ và S-
print(count_1)
# D_star = S_pos + S_neg

In [ ]:
# Thêm vào dataframe df_rating cột TV_new
df_rating['TV_new'] = TV_new
print(df_rating)

In [ ]:
# Tạo DataFrame mới để lưu các hàng thỏa mãn điều kiện
outliner = pd.DataFrame(columns=df_rating.columns)

# Lặp qua các hàng của DataFrame df_rating
for index, row in df_rating.iterrows():
    if row['TV_new'] != row['T_v']:
        # Thêm hàng vào DataFrame outliner
        outliner.loc[len(outliner)] = row

# In ra DataFrame outliner
print(outliner)

# Lưu DataFrame outliner vào file CSV
outliner.to_csv('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/outliner.csv', index=False)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# Load pre-trained model and tokenizer for Amazon reviews
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'  # Example model, you can choose other pre-trained models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)




In [ ]:
#Define the text to analyze
text = "I love the product"
def sentiment_analysis(text, model, tokenizer):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt')

    # Perform sentiment analysis
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted probabilities for each class
    probs = softmax(outputs.logits, dim=1).squeeze()

    # Convert the probabilities to human-readable labels
    labels = [1, 2, 3, 4, 5]  # For this model, it predicts star ratings
    predicted_label = labels[torch.argmax(probs).item()]
    # print(predicted_label)
    return predicted_label

# Print the result
print("Predicted sentiment:", sentiment_analysis(text, model, tokenizer))
print("Predicted sentiment:", torch.sigmoid(outputs.logits).squeeze()[1].item())
# print("Confidence:", probs[torch.argmax(probs)].item())

In [ ]:

predict_rating = df_rating['reviewText'].apply(lambda x: sentiment_analysis(x[:512], model=model, tokenizer=tokenizer))

predict_rating_list = predict_rating.tolist()
predict_rating_list =  defineRating(predict_rating_list)
print(predict_rating_list)

df_rating['predict_rating'] = predict_rating_list
print(df_rating)


In [ ]:
print(sentiment_analysis("The color pattern and fit is what I liked the most what I liked the least is that they are not easy to clean and stains do not come out very easy or at all", model, tokenizer))

In [ ]:
i = 0

for row in df_rating.itertuples():
    if row.predict_rating != row.T_v:
        i += 1
        print(row.predict_rating)
        print(row.T_v)
        print(row.overall)
        print(row.reviewText)
        print('------------------------')
print('count: ', i)

# Tạo DataFrame mới để lưu các hàng thỏa mãn điều kiện
outliner = pd.DataFrame(columns=['predict_rating', 'T_v', 'overall', 'reviewText'])
# Lặp qua các hàng của DataFrame df_rating
# Lặp qua các hàng của DataFrame df_rating
for index, row in df_rating.iterrows():
    if row['predict_rating'] != row['T_v']:
        # Thêm hàng vào DataFrame outliner
        outliner.loc[len(outliner)] = row

# In ra DataFrame outliner
print(outliner)

# Lưu DataFrame outliner vào file CSV
outliner.to_csv('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/outliner_bert.csv', index=False)

# BERT

In [82]:
import nltk
import math
import spacy
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from flair.data import Sentence
from flair.nn import Classifier
from flair.models import TextClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

pos_words = read_txt_file_to_list("./DeepCGSR_triet/data/positive-words.txt")
neg_words = read_txt_file_to_list("./DeepCGSR_triet/data/negative-words.txt")
print(pos_words)
print(neg_words)
a = ['JJ', 'JJR', 'JJS', 'RB', 'RBR', 'RBS']
brands = ['dove', 'aveda', 'silver']
keywords = ['overall', 'summary']

tagger = Classifier.load('pos')
# Load the spaCy English model
nlp = spacy.load("en_core_web_sm")
def remove_stopwords(sentence):
    """
    Loại bỏ các từ dừng từ câu sử dụng NLTK.

    Parameters:
    - sentence (str): Câu cần loại bỏ từ dừng.

    Returns:
    - str: Câu sau khi loại bỏ các từ dừng.
    """
    # Tách câu thành các từ
    words = word_tokenize(sentence)

    # Tải danh sách từ dừng từ NLTK
    stop_words = set(stopwords.words('english'))

    # Loại bỏ các từ dừng
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Ghép lại các từ thành câu mới
    filtered_sentence = ' '.join(filtered_words)

    return filtered_sentence

# df_rating['reviewText'] = df_rating['reviewText'].apply(remove_stopwords)

def get_sentiment_words(text):
    # Khởi tạo SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Tách câu thành các từ
    words = nltk.word_tokenize(text)

    # Lấy danh sách các từ ngừng (stop words)
    stop_words = set(stopwords.words('english'))

    # Danh sách các từ có cảm xúc
    sentiment_words = []

    # Kiểm tra từng từ trong câu
    for word in words:
        print(word, " - ", sia.polarity_scores(word)['compound'])
        # Nếu từ không phải là từ ngừng và có điểm tính cảm từ VADER lớn hơn 0
        # if word.lower() not in stop_words and sia.polarity_scores(word)['compound'] > 0:
        if word.lower() not in stop_words and sia.polarity_scores(word)['compound'] != 0:
            sentiment_words.append(word)

    return sentiment_words


def sentiment_analysis_flair(sentence):
    # Kiểm tra xem câu có rỗng không
    if not sentence.strip():
        return 0.0
    # Load pretrained sentiment model
    classifier = TextClassifier.load('en-sentiment')

    # Create a Flair Sentence
    flair_sentence = Sentence(sentence)

    # Predict sentiment
    classifier.predict(flair_sentence)
    # print(flair_sentence)
    # Get sentiment value (POSITIVE/NEGATIVE) and score
    sentiment_value = flair_sentence.labels[0].value
    sentiment_score = flair_sentence.labels[0].score
    score = 0.0
    bias = 1.0
    for keyword in keywords:
        if keyword.lower() in sentence.lower():
            bias = 1.75
    
    if(sentiment_value == 'POSITIVE'):
        score = sentiment_score * bias
    else:
        score = -sentiment_score * bias

    return score

#Define the text to analyze

def sentiment_analysis(text, model, tokenizer):
    # Tokenize the text
    # print(text)
    if not text:
        return -1
    text = text[:512]
    inputs = tokenizer(text, return_tensors='pt')

    # Perform sentiment analysis
    with torch.no_grad():
        outputs = model(**inputs)

    # Get the predicted probabilities for each class
    probs = softmax(outputs.logits, dim=1).squeeze()
    # print(probs)
    # Convert the probabilities to human-readable labels
    labels = [1, 2, 3, 4, 5]  # For this model, it predicts star ratings
    
    predicted_label = labels[torch.argmax(probs).item()]
    # print(predicted_label)
    return predicted_label

def check_string(input_value):
    # Kiểm tra xem giá trị có phải là chuỗi hay không
    if isinstance(input_value, str):
        # Nếu là chuỗi, trả về giá trị không thay đổi
        return input_value
    else:
        # Nếu không phải là chuỗi, trả về chuỗi rỗng
        return ''

def seperate_sentences(text):
    text = check_string(text)
    doc = nlp(text)
    # Khởi tạo một danh sách để lưu trữ các câu
    sentences = []
    current_sentence = ""
    # Duyệt qua mỗi từ trong văn bản
    for token in doc:
        # Nếu từ hiện tại không phải là dấu kết thúc câu, thêm từ vào câu hiện tại
        
        if token.is_sent_start:
            # Nếu có câu hiện tại, thêm câu vào danh sách câu
            if current_sentence:
                sentences.append(current_sentence.strip())
            # Bắt đầu một câu mới
            current_sentence = token.text
        elif token.text in ["but", "and"]:
            # print("that but:", current_sentence)
            if (brand.lower() in current_sentence.lower() for brand in brands):
                # print("need continue: ", current_sentence)
                current_sentence += " " + token.text
                continue
            elif current_sentence:
                sentences.append(current_sentence.strip())
                current_sentence = token.text
        else:
            # Thêm từ vào câu hiện tại
            current_sentence += " " + token.text
    # Thêm câu cuối cùng vào danh sách câu
    if current_sentence:
        sentences.append(current_sentence.strip())
    return sentences

def split_sentences_and_filter_sentiment(text):
    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Tách đoạn văn thành các câu nhỏ
    # sentences = nltk.sent_tokenize(text)
    sentences = seperate_sentences(text)
    if len(sentences) == 1 and not re.search(r'[.!?]', text):
        sentences = [text]
    # print(sentences)
    # Lọc các câu không chứa bất kỳ từ miêu tả cảm xúc nào
    filtered_sentences = []
    for sentence in sentences:
        # print("---- sentence ------")
        if not sentence.strip():
            continue
        sentence = sentence.strip()
        words = nltk.word_tokenize(sentence)
        
        # for word in words:
        #     if(sia.polarity_scores(word)['compound'] != 0 ):
        #         print(word, ":", sia.polarity_scores(word)['compound'])

        contains_sentiment = (any(sia.polarity_scores(word)['compound'] != 0 for word in words) or 
                              any(word.lower() in pos_words or word.lower() in neg_words for word in words))
        for word in words:
            if(word.lower() in brands):
                contains_sentiment = False
        # Flair
        
        sentence_flair = Sentence(sentence)
        tagger.predict(sentence_flair)
        # print(sentence_flair.tokens)
        is_a = False
        for token in sentence_flair.tokens:
            for label in token.get_labels('pos'):
                if label.value in a:
                    # print(token, label.value)
                    is_a = True
                    break
        for word in words:
            if(word.lower() in brands):
                contains_sentiment = False
                is_a = False
        #--------------
        if contains_sentiment:
            filtered_sentences.append(sentence)
        elif is_a:
            filtered_sentences.append(sentence)
        else:
            print("remove: ", sentence)
            # print(evaluate_sentiment_flair(sentence))
        # print("---- end sentence ------")
    # Thêm câu vào danh sách nếu không có câu nào chứa từ miêu tả cảm xúc
    if not filtered_sentences:
        filtered_sentences = [""]
    # print(filtered_sentences)
    return filtered_sentences

[nltk_data] Downloading package punkt to /home/triet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/triet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/triet/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


['a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable', 'ag

In [76]:
from transformers import BertTokenizer, BertForSequenceClassification
from torch.nn.functional import softmax
import torch

# Load pre-trained model and tokenizer for Amazon reviews
model_name = 'nlptown/bert-base-multilingual-uncased-sentiment'  # Example model, you can choose other pre-trained models
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/.venv/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [80]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

predictRating = []
text = "Luv this song!!"
# text = remove_stopwords(text)


sentences = split_sentences_and_filter_sentiment(text)
total_sentiment = 0
for sentence in sentences:
    # sentence = remove_stopwords(sentence)
    score = sentiment_analysis_flair(sentence)
    print(sentence, score)
    total_sentiment += score
    # print(total_sentiment)
print(total_sentiment)
if total_sentiment > 0.0:
    predictRating.append(1)
    print("Result: 1")
else:
    predictRating.append(-1)
    print("Result: -1")

[Token[0]: "Luv" → NNP (0.9986), Token[1]: "this" → DT (1.0), Token[2]: "song" → NN (0.9983), Token[3]: "!" → . (0.9984), Token[4]: "!" → . (0.9767)]
Sentence[5]: "Luv this song ! !" → POSITIVE (0.9994)
Luv this song ! ! 0.9994039535522461
0.9994039535522461
Result: 1


In [83]:
import tqdm
import os


predictRating = []
filtered_sentences = []
def process_and_save(df_rating, output_file):

    output_df = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText', 'filteredReview'])
    score = 0
    # Đoạn mã dùng tqdm để hiển thị tiến trình %
    for i, row in tqdm.tqdm(df_rating.iterrows(), desc="Processing", total=len(df_rating)):
        sentences = split_sentences_and_filter_sentiment(row['reviewText'])
        filtered_sentences.append(" ".join(sentences))
        total_sentiment = sum(sentiment_analysis_flair(sentence) for sentence in sentences)
        if total_sentiment > 0.0:
            score = 1
        else:
            score = -1
        predictRating.append(score)
        # Lưu vào DataFrame và ghi vào file CSV sau mỗi 10 record
        output_df.loc[len(output_df)] = { 'reviewerID': row['reviewerID'],
                                          'asin': row['asin'],
                                          'predictRating': score,
                                          'T_v': row['T_v'],
                                          'overall': row['overall'],
                                          'reviewText': row['reviewText'],
                                          'filteredReview': " ".join(sentences) }
        
        if (i + 1) % 5 == 0:
            output_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)
            output_df = pd.DataFrame(columns=['reviewID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText'])
    
    # Ghi phần còn lại của DataFrame vào file CSV
    if not output_df.empty:
        output_df.to_csv(output_file, mode='a', header=not os.path.exists(output_file), index=False)


# Sử dụng hàm
process_and_save(df_rating, '/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/output_DigitalMusic.csv')


Processing:   0%|          | 0/10000 [00:00<?, ?it/s]

remove:  One of my summer anthems .


Processing:   0%|          | 2/10000 [00:08<11:16:32,  4.06s/it]

remove:  Smoochiesss .....


Processing:   0%|          | 3/10000 [00:10<9:03:19,  3.26s/it] 

...

remove:  I listen to this all the time in my Ipod


Processing:   1%|          | 83/10000 [04:34<7:24:05,  2.69s/it] 

remove:  I gave this five stars for my Wife .
remove:  I bought this version for her .


Processing:   1%|          | 86/10000 [04:49<9:17:09,  3.37s/it] 


KeyboardInterrupt: 

In [12]:
output = pd.read_csv('./DeepCGSR_triet/output.csv')
output_df = pd.DataFrame(output)
print("Length Original Dataset: ", len(output_df))
outliner = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText'])
filteredRecords = pd.DataFrame(columns=['reviewerID', 'asin', 'predictRating', 'T_v', 'overall', 'reviewText'])
for i, row in output_df.iterrows():
    if row['predictRating'] != row['T_v']:
        outliner.loc[len(outliner)] = row
    else:
        filteredRecords.loc[len(filteredRecords)] = row
        # print('------------------------')
outliner.to_csv('./DeepCGSR_triet/outliner_DM.csv', index=False)
filteredRecords.to_csv('./DeepCGSR_triet/filteredRecords_DM.csv', index=False)

print("Length Outliner Dataset: ", len(outliner))
print("Length Filered Dataset: ", len(filteredRecords))

Length Original Dataset:  10010
Length Outliner Dataset:  944
Length Filered Dataset:  9066


In [19]:
import pandas as pd
import json
from tqdm import tqdm


#Load json data to dataframe
raw_data = pd.read_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/Digital_Music_5.json', lines=True)
print("Dataset: ", len(raw_data))
# Loại bỏ các dòng có reviewText trùng lặp
# all_beauty_data = remove_duplicate_rows(all_beauty_data)
# print("Dataset remover duplication: ", len(all_beauty_data))
# raw_data['reviewText'] = filtered_sentences
# raw_data['T_v'] = T_v
# raw_data['predictRating'] = predictRating
# Kiểm tra các bản ghi trong all_beauty_data mà không có trong df_rating
count = 0
result_data = raw_data.copy(deep=False)
result_data = result_data.iloc[0:0]

for filter_index, filter_row in tqdm(filteredRecords.iterrows(), total=len(filteredRecords), desc="Processing"):
    for index, row in raw_data.iterrows():
        if (row['reviewerID'] == filter_row['reviewerID'] and row['asin'] == filter_row['asin'] 
            and row['reviewText'] == filter_row['reviewText']):
            result_data.loc[len(result_data)] = row
            count += 1
            # print(count, row['asin'], row['reviewerID'], row['reviewText'])
            break

print("Filtered dataset: ", len(result_data))
result_data.to_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/Digital_Music_Filtered.json', orient='records', lines=True)

data_filtered = pd.read_json('/home/triet/ComputerScience_K32/Recommendation_System/DeepCGSR_triet/data/raw/Digital_Music_Filtered.json', lines=True)



Dataset:  169781


Processing: 100%|██████████| 9066/9066 [28:25<00:00,  5.32it/s]  


Filtered dataset:  8957
